[View in Colaboratory](https://colab.research.google.com/github/Giffy/CarCrashDetector/blob/master/4_Prediction.ipynb)

# Index
<ol>
    <li><a href="#system _setup">System setup</a>
    <li><a href="#load_train">Load pre-trained ResNet</a>
    <li><a href="#pred_batch">Prediction batch processing</a>
    

<br>
<a id="system_setup"> </a>
# 1. System setup

## 1.1 Link Goggle Drive with Colab 

Run the code and follow the link to get an authentification key, copy it and paste in the box that will appear in jupyter notebook.  After fist key the script will ask for a second authentification key, follow the process as above. 

Source : https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d 


In [0]:
# Check is Link to Drive is OK
google = !if [ -d 'mydrive/' ]; then echo "1" ; else echo "0"; fi
if (google[0] is '0' ):
  !apt-get install -y -qq software-properties-common python-software-properties module-init-tools > /dev/null
  !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
  !apt-get update -qq 2>&1 > /dev/null
  !apt-get -y install -qq google-drive-ocamlfuse fuse > /dev/null
  from google.colab import auth
  auth.authenticate_user()
  from oauth2client.client import GoogleCredentials
  creds = GoogleCredentials.get_application_default()
  import getpass
  !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
  vcode = getpass.getpass()
  !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
  # Makes folder 'mydrive' in colab and mounts a link to 'Google drive'
  !mkdir -p mydrive
  !google-drive-ocamlfuse mydrive
!if [ -d 'mydrive/' ]; then echo "Connection to Google drive successful" ; else echo "Error to connect to Google drive"; fi

## 1.2 Install and update python libraries
Set up of Python, installing the required modules
<ol>
   <li>Updated python package manager (pip)
   <li>Torchvision
   <li>Pillow 4.0.0 (required for fastai library)
   <li>Image
   <li>Fast ai

In [0]:
!pip install --upgrade pip  > /dev/null
!pip install http://download.pytorch.org/whl/cu75/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl  && pip install torchvision > /dev/null
!pip install Pillow==4.0.0 > /dev/null
!pip install PIL > /dev/null
!pip install image > /dev/null
!pip install fastai > /dev/null
!apt update && apt install -y libsm6 libxext6 > /dev/null

<br>
<a id="load_train"> </a>
# 2. Load pre-trained ResNet Model

## 2.1 Libraries setup
Import the fastai libraries

In [0]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
# This file contains all the main external libs we'll use
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
import sys

## 2.2 Load model


In [0]:
###   Add here the name of the model to test
model_name = "carCrash180614-1043_sz299_bs60_ep5"

In [0]:
arch=resnet34
sz = 299                                                                        # Type same size used during model creation. If you donwloaded the model, a new precomputed activations will be generated.
batch_size = 60                                                                 # Type same batch_size used during model creation. If you donwloaded the model, a new precomputed activations will be generated.
PATH = "mydrive/CarCrashDetection/Dataset/"
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz), bs=batch_size)     
learn = ConvLearner.pretrained(arch, data, precompute=True)


print ( "Loading model:", model_name,".h5\n")
learn.load(model_name)
print ( "Model loaded")

## 2.3 Load image to test

In [0]:
PATH = "mydrive/CarCrashDetection/Dataset/train/no_accident/"
fn = "4frame60.png"

PATH = "mydrive/CarCrashDetection/Dataset/train/accident/"
fn = "frame0020.png"

img = open_image(PATH+fn)

print(img.shape)
plt.imshow(img);

## 2.4 Prediction Test

In [0]:
trn_tfms, val_tfms = tfms_from_model(arch,sz)      # get transformations
im = val_tfms (img)

learn.precompute=False                             # We'll pass in a raw image, not activations
preds = learn.predict_array(im[None])
np.argmax(preds)                                   # preds are log probabilities of classes

<br>
<a id="pred_batch"> </a>
# 3. Prediction Batch processing Test

## 3.1 Define test bench folder to check

In [0]:
#PATH = "mydrive/CarCrashDetection/video_to_check/frames1-noaccident/"
#PATH = "mydrive/CarCrashDetection/video_to_check/frames4-noaccident/"
#PATH = "mydrive/CarCrashDetection/video_to_check/frames5-noaccident/"
PATH = "mydrive/CarCrashDetection/video_to_check/frames9-noaccident/"
#PATH = "mydrive/CarCrashDetection/video_to_check/frames10-noaccident/"

#PATH = "mydrive/CarCrashDetection/video_to_check/frames2-accident/"
#PATH = "mydrive/CarCrashDetection/video_to_check/frames3-accident/"
#PATH = "mydrive/CarCrashDetection/video_to_check/frames6-accident/"
#PATH = "mydrive/CarCrashDetection/video_to_check/frames7-accident/"
#PATH = "mydrive/CarCrashDetection/video_to_check/frames8-accident/"


filenames_to_check = !ls -tr {PATH} 
number_images = len(filenames_to_check)
print(f"Number of images to check: {number_images}")

## 3.2 Prediction

In [0]:
count = 0
list = []
for i in range(number_images):
  fn = filenames_to_check[i]
  img = open_image(PATH+fn)
  trn_tfms, val_tfms = tfms_from_model(arch,sz)      
  im = val_tfms (img)

  learn.precompute=False                             
  preds = learn.predict_array(im[None])
  p = np.argmax(preds)       
  list.append(p)
  count += 1 
  if (count%5):    
    b = round((count*100 / number_images),0)
    sys.stdout.write('\r'+ str(b) + "%")

print ("")
print ("Imagenes procesadas :", count)
print ("Tentative number of no accidents :", list.count(1))
print ("Tentative number of accidents :", list.count(0))

## 3.3 Prediction normalization

In [0]:
# normalization setup
seg_lenght = 5      # Segment lenght to analize
steps = 4           # number of steps between length
positivos = 2       # number of '0's in the lenght

In [0]:
list_normalized = []


for i in range(0, (len(list)-(steps+1)), steps):                 # iterates the prediction list according the steps
  norm = 0
  for j in range (seg_lenght):
    posicion = i+j
    norm = norm + list[posicion]

  if (norm>positivos):
    list_normalized.append(1)
  else:
    list_normalized.append(0)

In [0]:
print (list)
print (list_normalized)

## 3.4 Resultado

In [0]:
count = 0
accidente = 0
for i in range( len (list_normalized)):
  if (count == 3):
    accidente = 1
    break
  else:
    if (list_normalized[i] == 1):
      count == 0
    else:
      count +=1
if (accidente == 0):
  print ("No accident in video")
else:
  print ("Warning, accident")    